Aim: The aim of this notebook is to explore the Hellinger Distance dependency on the basic parameters of IidPartitioner and Dirichlet Partitioner when using CIFAR10 dataset.

# Imports

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys
import os
print(os.getcwd())
# you're in fl-heterogeneity/heterogeneity/notebooks
sys.path.append(os.path.abspath("./../.."))

In [4]:
import itertools

import numpy as np
import pandas as pd
from flwr_datasets import FederatedDataset
from flwr_datasets.partitioner import IidPartitioner, DirichletPartitioner, ShardPartitioner, InnerDirichletPartitioner

from heterogeneity.metrics.hellinger_distance import compute_hellinger_distance
from heterogeneity.utils import create_lognormal_partition_sizes

# CIFAR10

## IID

In [47]:
# Sample usage
num_partitions = 10
iid_partitioner = IidPartitioner(num_partitions=num_partitions)
cifar_iid = FederatedDataset(dataset="cifar10", partitioners={"train" : iid_partitioner})
cifar_iid_partitions = [cifar_iid.load_partition(i) for i in range(num_partitions)]
# Basic statistics of the global train CIFAR10 data
train = cifar_iid.load_split("train")
train_labels = train["label"]
pd.Series(train_labels).value_counts().sort_index()

In [48]:
num_partitions_to_cifar_iid_partitions = {}
num_partitions_to_cifar_iid_fds = {}
num_partitions_list = [3, 10, 30, 100, 300, 1000]
for num_partitions in num_partitions_list:
    iid_partitioner = IidPartitioner(num_partitions=num_partitions)
    cifar_iid = FederatedDataset(dataset="cifar10", partitioners={"train" : iid_partitioner})
    num_partitions_to_cifar_iid_fds[num_partitions] = cifar_iid
    # cifar_iid_partitions = [cifar_iid.load_partition(i) for i in range(num_partitions)]
    # num_partitions_to_cifar_iid_partitions[num_partitions] = cifar_iid_partitions

num_partitions_to_cifar_iid_hellinger_distance = {}
num_partitions_to_cifar_iid_hellinger_distance_list = {}
for num_partitions, cifar_iid_fds in num_partitions_to_cifar_iid_fds.items():
    hellinger_distance_list, avg_hellinger_distance = compute_hellinger_distance(cifar_iid_fds.partitioners["train"])
    num_partitions_to_cifar_iid_hellinger_distance_list[num_partitions] = hellinger_distance_list
    num_partitions_to_cifar_iid_hellinger_distance[num_partitions] = avg_hellinger_distance


In [104]:
iid_helinger_results = pd.Series(num_partitions_to_cifar_iid_hellinger_distance, name="iid_helinger").to_frame().style.background_gradient()
iid_helinger_results.index.name = "num_partitions"
iid_helinger_results

Is it desired to that degree? When the num_partitions is 1000, then each partition is of size 5. That can make it indeed heterogeneus.

# Dirichlet

In [108]:
# Example usage
num_partitions = 10
alpha = [0.1] * 10
dirichlet_partitioner = DirichletPartitioner(num_partitions=num_partitions, alpha=alpha, partition_by="label")
cifar_dir = FederatedDataset(dataset="cifar10", partitioners={"train" : dirichlet_partitioner})
cifar_dir_partitions = [cifar_dir.load_partition(i) for i in range(num_partitions)]

In [109]:
num_partitions_to_cifar_dir_partitions = {}
num_partitions_to_cifar_dir_fds = {}
num_partitions_list = [3, 10, 30, 100, 300, 1000]
alpha_list = [0.1, 0.3, 1., 3., 10., 100., 100.]
for num_partitions, alpha in itertools.product(num_partitions_list, alpha_list):
    dir_partitioner =  DirichletPartitioner(num_partitions=num_partitions, alpha=alpha, partition_by="label", self_balancing=False)
    cifar_dir = FederatedDataset(dataset="cifar10", partitioners={"train" : dir_partitioner})
    num_partitions_to_cifar_dir_fds[(num_partitions, alpha)] = cifar_dir
    # cifar_iid_partitions = [cifar_iid.load_partition(i) for i in range(num_partitions)]
    # num_partitions_to_cifar_iid_partitions[num_partitions] = cifar_iid_partitions

num_partitions_to_cifar_dir_hellinger_distance_list = {}
num_partitions_to_cifar_dir_hellinger_distance = {}
for (num_partitions, alpha), cifar_dir_fds in num_partitions_to_cifar_dir_fds.items():
    print((num_partitions, alpha))
    try:
        hellinger_distance_list, avg_hellinger_distance = compute_hellinger_distance(cifar_dir_fds.partitioners["train"])
    except:
        print(f"Sampling failed for {(num_partitions, alpha)}")
        hellinger_distance_list, avg_hellinger_distance = np.nan, np.nan
    num_partitions_to_cifar_dir_hellinger_distance_list[(num_partitions, alpha)] = hellinger_distance_list
    num_partitions_to_cifar_dir_hellinger_distance[(num_partitions, alpha)] = avg_hellinger_distance

In [110]:
hel_dir = pd.Series(num_partitions_to_cifar_dir_hellinger_distance).unstack(level=1)#.style.background_gradient(axis=None)
hel_dir.index.name = "num_partitions"
hel_dir.columns.name = "alpha"
hel_dir.style.background_gradient(axis=None)

In [44]:
num_partitions_to_cifar_dir_partitions = {}
num_partitions_to_cifar_dir_fds = {}
num_partitions_list = [3, 10, 30, 100, 300, 1000]
alpha_list = [0.1, 0.3, 1., 3., 10., 100., 100.]
for num_partitions, alpha in itertools.product(num_partitions_list, alpha_list):
    dir_partitioner =  DirichletPartitioner(num_partitions=num_partitions, alpha=alpha, partition_by="label", self_balancing=True)
    cifar_dir = FederatedDataset(dataset="cifar10", partitioners={"train" : dir_partitioner})
    num_partitions_to_cifar_dir_fds[(num_partitions, alpha)] = cifar_dir
    # cifar_iid_partitions = [cifar_iid.load_partition(i) for i in range(num_partitions)]
    # num_partitions_to_cifar_iid_partitions[num_partitions] = cifar_iid_partitions

num_partitions_to_cifar_dir_hellinger_distance_list = {}
num_partitions_to_cifar_dir_hellinger_distance = {}
for (num_partitions, alpha), cifar_dir_fds in num_partitions_to_cifar_dir_fds.items():
    print((num_partitions, alpha))
    try:
        hellinger_distance_list, avg_hellinger_distance = hellinger_distance(cifar_dir_fds, "train")
    except:
        print(f"Sampling failed for {(num_partitions, alpha)}")
        hellinger_distance_list, avg_hellinger_distance = np.nan, np.nan
    num_partitions_to_cifar_dir_hellinger_distance_list[(num_partitions, alpha)] = hellinger_distance_list
    num_partitions_to_cifar_dir_hellinger_distance[(num_partitions, alpha)] = avg_hellinger_distance

In [45]:
pd.Series(num_partitions_to_cifar_dir_hellinger_distance).to_frame().unstack(level=1).style.background_gradient(axis=None)

The self_balancing (size balancing) creates more heterogenous dataset division.

## Shard

In [57]:
params_to_partitioner = {}
num_partitions_list = [3, 10, 30, 100, 300, 1000]
num_shards_per_partition_list = [2, 3, 4, 5]
for num_partitions, num_shards_per_partition in itertools.product(num_partitions_list, num_shards_per_partition_list):
    partitioner = ShardPartitioner(num_partitions=num_partitions, partition_by="label", num_shards_per_partition=num_shards_per_partition)
    fds = FederatedDataset(dataset="cifar10", partitioners={"train" : partitioner})
    params_to_partitioner[(num_partitions, num_shards_per_partition)] = fds

parameters_to_shard_cifar_fds_metric_list = {}
parameters_to_shard_cifar_fds_metric = {}
for (num_partitions, num_shards_per_partition), fds in params_to_partitioner.items():
    print((num_partitions, num_shards_per_partition))
    try:
        metric_list, avg_metric = compute_hellinger_distance(fds.partitioners["train"])
    except:
        print(f"Sampling failed for {(num_partitions, num_shards_per_partition)}")
        metric_list, avg_metric = np.nan, np.nan
    parameters_to_shard_cifar_fds_metric_list[(num_partitions, num_shards_per_partition)] = metric_list
    parameters_to_shard_cifar_fds_metric[(num_partitions, num_shards_per_partition)] = avg_metric

In [103]:
shard_emd_results = pd.Series(parameters_to_shard_cifar_fds_metric).unstack(level=1)
shard_emd_results.index.name = "num_partitions"
shard_emd_results.columns.name = "num_shards"
shard_emd_results.style.background_gradient(axis=None)

In [75]:
shard_emd_results

## Inner Dirichlet

In [5]:

dataset_name = "cifar10"
# num_partitions = 10
# sigma = 0.3
# partition_sizes = create_lognormal_partition_sizes(dataset_name, num_partitions, sigma)
# 
# alpha = 0.1
# dirichlet_partitioner = InnerDirichletPartitioner(partition_sizes=partition_sizes, partition_by="label", alpha=0.1)
# cifar_dir = FederatedDataset(dataset="cifar10", partitioners={"train" : dirichlet_partitioner})
# cifar_dir_partitions = [cifar_dir.load_partition(i) for i in range(num_partitions)]

num_partitions_to_cifar_dir_fds = {}
num_partitions_list = [3, 10, 30, 100, 300, 1000]
alpha_list = [0.1, 0.3, 1., 3., 10., 100., 100.]
sigma_list = [0.1, 0.3, 1., 3.]
partition_sizes_dict = {}
for num_partitions, alpha, sigma in itertools.product(num_partitions_list, alpha_list, sigma_list):
    partition_sizes = create_lognormal_partition_sizes(dataset_name, num_partitions, sigma)
    dir_partitioner =  InnerDirichletPartitioner(partition_sizes=partition_sizes, partition_by="label", alpha=alpha)
    cifar_dir = FederatedDataset(dataset="cifar10", partitioners={"train" : dir_partitioner})
    num_partitions_to_cifar_dir_fds[(num_partitions, alpha, sigma)] = cifar_dir
    partition_sizes_dict[(num_partitions, alpha, sigma)] = partition_sizes
    # cifar_iid_partitions = [cifar_iid.load_partition(i) for i in range(num_partitions)]
    # num_partitions_to_cifar_iid_partitions[num_partitions] = cifar_iid_partitions

num_partitions_to_cifar_dir_metric_list = {}
num_partitions_to_cifar_dir_metric = {}
for (num_partitions, alpha, sigma), cifar_dir_fds in num_partitions_to_cifar_dir_fds.items():
    print((num_partitions, alpha, sigma))
    try:
        metric_list, avg_metric = compute_hellinger_distance(cifar_dir_fds.partitioners["train"])
    except:
        print(f"Sampling failed for {(num_partitions, alpha, sigma)}")
        metric_list, avg_metric = np.nan, np.nan
    num_partitions_to_cifar_dir_metric_list[(num_partitions, alpha, sigma)] = metric_list
    num_partitions_to_cifar_dir_metric[(num_partitions, alpha, sigma)] = avg_metric

In [13]:
emd_dir = pd.Series(num_partitions_to_cifar_dir_metric, name="emd_inner_dir").unstack(level=1)
emd_dir

In [16]:
partition_sizes_dict[(10, 0.3, 0.1)]

In [17]:
partition_sizes_dict[(10, 0.3, 3)]

# FEMNIST

In [3]:
import pandas as pd
path = "../../results/adamnarozniak_old/femnist/IidPartitioner/compute_hellinger_distance.csv"
femnist_helinger = pd.read_csv(path)
femnist_helinger = femnist_helinger.drop_duplicates()
femnist_helinger.groupby(["num_partitions"])["metric_value"].mean().to_frame().style.background_gradient(axis=None)

,metric_value
num_partitions,
3,0.137941
10,0.166024
30,0.170423
100,0.175127
300,0.180012
1000,0.200157


In [31]:
path = "../../results/adamnarozniak/femnist/DirichletPartitioner/compute_hellinger_distance.csv"
femnist_helinger = pd.read_csv(path)
femnist_helinger = femnist_helinger.drop_duplicates()
femnist_helinger.groupby(["num_partitions", "alpha"])["metric_value"].mean().to_frame().unstack(level=-1).style.background_gradient(axis=None)

In [34]:
femnist_helinger

# VISUALIZE RESULTS

In [ ]:
import pandas as pd
from IPython.display import display

In [12]:
partitioner_name = "DirichletPartitioner"
for ds_name in ["cifar10", "cifar100", "adamnarozniak/femnist"]:
    path = f"../../results/{ds_name}/{partitioner_name}/compute_hellinger_distance.csv"
    metrics = pd.read_csv(path)
    metrics = metrics.drop_duplicates()
    print(ds_name)
    display(metrics.groupby(["num_partitions", "alpha"])["metric_value"].mean().to_frame().unstack(level=-1).style.background_gradient(axis=None))

cifar10


cifar100


adamnarozniak/femnist


In [33]:
partitioner_name = "ClassConstrainedPartitioner"
for ds_name in ["cifar10", "cifar100", "adamnarozniak/femnist"]:
    path = f"../../results/{ds_name}/{partitioner_name}/compute_hellinger_distance.csv"
    metrics = pd.read_csv(path)
    metrics = metrics.drop_duplicates()
    print(ds_name)
    if ds_name == "cifar100":
        display(metrics.groupby(["num_partitions", "num_classes_per_partition"])["metric_value"].mean().to_frame().unstack(level=-1).iloc[:, 4:].style.background_gradient(axis=None))
    elif ds_name == "adamnarozniak/femnist":
        display(metrics.groupby(["num_partitions", "num_classes_per_partition"])["metric_value"].mean().to_frame().unstack(level=-1).iloc[:, 4:].style.background_gradient(axis=None))
    else:
        display(metrics.groupby(["num_partitions", "num_classes_per_partition"])["metric_value"].mean().to_frame().unstack(level=-1).style.background_gradient(axis=None))

cifar10


cifar100


adamnarozniak/femnist
